In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
from dataclasses import dataclass
from copy import deepcopy

In [3]:
# Load the concrete data
df = pd.read_csv("data/concrete_data.csv")

# Display the data
df

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [4]:
# Separate into train and test datasets

train_df = df.sample(frac=0.7, random_state=42) # random state ensures we always get the same sample
test_df = df.drop(train_df.index)
train_df.columns

Index(['cement', ' blast_furnace_slag', ' fly_ash', ' water',
       ' superplasticizer', ' coarse_aggregate', ' fine_aggregate', ' age',
       ' concrete_compressive_strength'],
      dtype='object')

In [5]:
# copy the target column from the input data into separate numpy arrays

train_targets = train_df[' concrete_compressive_strength'].to_numpy()
test_targets = test_df[' concrete_compressive_strength'].to_numpy()

test_targets

array([61.89, 44.3 , 42.33, 47.81, 41.84, 28.24, 52.12, 41.72, 53.69,
       38.41, 50.46, 40.76, 33.12, 50.95,  9.87, 48.7 , 33.4 , 28.6 ,
       24.4 , 35.3 , 49.2 , 55.6 , 54.1 , 56.1 , 68.3 , 66.9 , 60.29,
       68.5 , 71.3 , 74.7 , 71.3 , 49.9 , 60.2 , 64.3 , 55.2 , 66.1 ,
       72.99, 79.4 , 77.3 , 59.89, 62.5 , 57.6 , 67.8 , 24.89, 29.45,
       10.38, 22.84, 33.96, 21.06, 26.4 , 35.34, 20.92, 24.9 , 28.47,
       38.56, 10.76,  7.75, 30.39, 50.77, 53.9 , 22.32, 24.54, 31.35,
       40.86, 30.23, 29.22, 38.33, 42.92, 44.4 , 45.08, 15.44, 26.77,
       45.84, 29.65, 13.12, 36.64, 45.37, 48.67, 23.51, 39.15, 55.64,
       52.04, 33.36, 44.14, 42.29, 42.22, 56.85, 21.91, 56.74, 33.73,
       46.64, 50.08, 66.95, 52.2 , 46.23, 31.97, 43.06, 67.57, 41.37,
       60.28, 56.83, 51.02, 44.13, 55.65, 47.28, 29.16, 67.87, 58.52,
       53.58, 14.4 , 21.29, 15.82, 12.55,  8.49, 11.98, 19.42, 41.41,
       27.22, 39.64, 51.26, 55.02, 49.99, 53.66, 56.06, 33.56, 57.03,
       44.42, 53.39,

In [6]:
# drop the target column from the input data to create input feature arrays

train_features = train_df.drop(columns=[' concrete_compressive_strength']).to_numpy()
test_features = test_df.drop(columns=[' concrete_compressive_strength']).to_numpy()

test_features

array([[ 540. ,    0. ,    0. , ..., 1055. ,  676. ,   28. ],
       [ 198.6,  132.4,    0. , ...,  978.4,  825.5,  360. ],
       [ 190. ,  190. ,    0. , ...,  932. ,  670. ,   90. ],
       ...,
       [ 159.8,  250. ,    0. , ..., 1049.3,  688.2,   28. ],
       [ 166. ,  259.7,    0. , ...,  858.8,  826.8,   28. ],
       [ 322.2,    0. ,  115.6, ...,  817.9,  813.4,   28. ]],
      shape=(309, 8))

In [7]:
# create a layer class for the MLP

class LayerMLP:
    def __init__(self, size_input: int, size_hidden: int):
        # Initialize weights and biases
        self.bias = np.full((size_hidden, 1), 0.01)
        self.weights = np.random.randn(size_hidden, size_input)

    def sigmoid(self, z):
        z_clip = np.clip(z, -500, 500)  # Prevent overflow
        return 1/(1 + np.exp(-z_clip))

    def forward(self, X):
        z = np.dot(self.weights, X) + self.bias
        a = self.sigmoid(z)
        return a
    
# Test the LayerMLP class
# layer = LayerMLP(size_input=5, size_hidden=3)
# print(layer.bias)
# print(layer.weights)

# X_sample = np.random.randn(1, 5).T
# print(X_sample)
# output = layer.forward(X_sample)
# print(output)

In [ ]:
# create a MLP class

class MLP:
    def __init__(self, size_input: int, size_hidden_layers: List[int]):
        self.layers: List[LayerMLP] = self.initialize_layers(size_input, size_hidden_layers)
        self.velocity_layers: List[LayerMLP] = self.initialize_layers(size_input, size_hidden_layers)  # for PSO velocity

    def initialize_layers(self, size_input: int, size_hidden_layers: List[int]):
        layers: List[LayerMLP] = []
        layers.append(LayerMLP(size_input=size_input, size_hidden=size_hidden_layers[0])) # first hidden layer

        size_layers = size_hidden_layers + [1]  # add output layer with one neuron
        for i in range(len(size_layers) - 1): # remaining hidden layers and output layer
            layer = LayerMLP(size_input=size_layers[i], size_hidden=size_layers[i+1])
            layers.append(layer)
        return layers

    def get_loss(self, prediction, target):
        # loss = np.mean((prediction.flatten() - target) ** 2)
        # return loss
        pass

    def forward(self, X):
        a = X
        for layer in self.layers:
            a = layer.forward(a)
        return a

# Test the MLP class
mlp = MLP(size_input=5, size_hidden_layers=[3, 4])
X_sample = np.random.randn(1, 5).T
output = mlp.forward(X_sample)
print(output)
    

[[0.64031611]]


In [ ]:
# Creation of AI class for training the MLP with our PSO

@dataclass
class AccelerationCoefficients:
    inertia_weight: float
    cognitive_weight: float
    social_weight: float
    global_best_weight: float
    jump_size: float

class Particle:
    def __init__(self, mlp: MLP, accel_coeff: AccelerationCoefficients):
        self.accel_coeff = accel_coeff
        # Initialize other attributes like position, velocity, personal best, etc.
        self.position = mlp
        self.personal_best = None
        self.informants = []
        self.fitness: float = 0.0

    def evaluate_fitness(self, X, target):
        self.prediction = self.mlp.forward(X)
        self.fitness = self.mlp.get_loss(self.prediction, target)

    def update_velocity(self, global_best):
        # Implement velocity update logic
        pass

    def update_position(self):
        pass

class ParticleSwarmOptimisation:
    def __init__(
            self,
            X: np.ndarray[tuple[int, int]],
            swarm_size: int,
            epochs: int,
            accel_coeff: AccelerationCoefficients,
            num_informants: int,
            mlp_size_hidden_layers: List[int]
        ):
        self.epochs = epochs
        self.accel_coeff = accel_coeff
        self.num_informants = num_informants

        self.population: List[Particle] = []
        for _ in range(swarm_size):
            mlp = MLP(size_input=X.shape[1], size_hidden_layers=mlp_size_hidden_layers)
            self.population.append(Particle(mlp=mlp, accel_coeff=accel_coeff))

        self.global_best = None

    def update_informants(self):
        for particle in self.population:
            particle.informants = np.random.choice(self.population, size=self.num_informants, replace=False).tolist()

    def update_global_best(self):
        for particle in self.population:
            particle.evaluate_fitness(X=train_features, y=train_targets)
            if self.global_best == None or particle.fitness < self.global_best.fitness:
                self.global_best = deepcopy(particle)

    def update_velocities(self):
        for particle in self.population:
            particle.update_velocity(self.global_best)

    def update_positions(self):
        for particle in self.population:
            particle.update_position()

    def train(self):
        self.update_informants()
        for epoch in range(self.epochs):
            self.update_global_best()
            self.update_velocities()
            self.update_positions()

pso = ParticleSwarmOptimisation(
    X=train_features,
    swarm_size=30,
    epochs=100,
    accel_coeff=AccelerationCoefficients(
        inertia_weight=0.5,
        cognitive_weight=1.5,
        social_weight=1.5,
        global_best_weight=1.0,
        jump_size=0.1,
    ),
    num_informants=5,
    mlp_size_hidden_layers=[5, 5]
)